#### [Naver 리뷰 데이터셋 전처리]<hr>
- trian : https://github.com/e9t/nsmc/blob/master/ratings_train.txt
- test : https://github.com/e9t/nsmc/blob/master/ratings_test.txt


[1] 데이터 준비 <hr>

In [53]:
### 데이터 다운로드
from urllib.request import urlretrieve

file_train = '../data/ratig_train.txt'
train_url = 'https://github.com/e9t/nsmc/raw/master/ratings_train.txt'
ret_train = urlretrieve(train_url, file_train)  # 파일 이름과 객체 반환

In [54]:
file_test = '../data/ratig_test.txt'
test_url = 'https://github.com/e9t/nsmc/raw/master/ratings_test.txt'
ret_test = urlretrieve(test_url, file_test)

In [55]:
import pandas as pd

trainDF = pd.read_table(file_train)

In [56]:
testDF = pd.read_table(file_test)

In [57]:
trainDF.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


[2] 데이터 전처리<hr>
- 결측치, 중복치
- 클래스 균형 데이터 여부
- 텍스트 데이터 전처리 => 정제(불용어, 노이즈데이터 제거), 토큰화, 정제, 단어사전 
    * 정제는 수시로 발생할 수 있음
- 텍스트 데이터 인코딩
- 텍스트 데이터 패딩

[2-1] 결측치, 중복치 확인 및 처리

In [58]:
trainDF.drop(labels='id', axis=1, inplace=True)

In [59]:
# 중복치 3659 => 제거
sum(trainDF.duplicated())

trainDF.drop_duplicates(inplace=True)

# 제거 후 확인
sum(trainDF.duplicated())

0

In [60]:
# document col에 결측치 5개 존재 => 제거
trainDF.isna().sum()

trainDF.dropna(subset='document', inplace=True)

In [61]:
# 균형적인 데이터
trainDF.value_counts(subset='label')

label
0    73429
1    72910
Name: count, dtype: int64

[2-2] 텍스트 데이터 전처리 : 정제(노이즈 데이터, 불용어 제거), 토큰화, 정수화
- 노이즈 데이터 기준 : 한글만, 한글+영어, 한글+영어+숫자, 한글+영어+특수문자
    * 노이즈 데이터 제거는 정규 표현식 이용
- 어근 추출과 표제어 추출 작업을 해야한다면 해당 기능을 포함하는 형태소 분석기를 사용해야함

In [62]:
import string

# regex = True : 정규식 적용

# 구두점 제거 : 정규표현식 [] => []내에 있는 문자들 중 하나라도 있으면~
trainDF['document'].replace(r'[{}]'.format(string.punctuation), '', regex=True, inplace=True)

In [94]:
# 한글과 공백을 제외하고 모두 제거
# [^ㄱ-ㅎㅏ-ㅣ가-힣 ] :  한글 자모와 완성형 한글, 그리고 공백을 제외한 나머지 모든 문자
hangule_patten = '[^가-힣 ]' 
trainDF['document'].replace(hangule_patten, '', regex=True, inplace=True)

In [95]:
# 공백 제거
import numpy as np
trainDF['document'].replace('\s+', np.NaN, regex=True, inplace=True)
trainDF.dropna(subset=['document'], inplace=True)

In [96]:
# 한글 불용어 제거
# stopword_path = '../data/hangul_stopword.txt'

# hangul_stopword = 'https://gist.githubusercontent.com/chulgil/d10b18575a73778da4bc83853385465c/raw/a1a451421097fa9a93179cb1f1f0dc392f1f9da9/stopwords.txt'

# urlretrieve(hangul_stopword, '../data/hangul_stopword.txt')

('../data/hangul_stopword.txt', <http.client.HTTPMessage at 0x1439d1ec400>)

In [109]:
# 한글 불용어 text 파일에 내용 추가하기

# with open('../data/stopword.txt', "r", encoding="utf-8") as source_file:
#     additional_content = source_file.read()

# # 대상 파일에 내용 추가
# with open('../data/hangul_stopword.txt', "a", encoding="utf-8") as target_file:
#     target_file.write(additional_content)

In [110]:
# 한글 불용어 text 내용 리스트에 담기
with open('../data/hangul_stopword.txt', 'r', encoding='utf-8') as file:
    stopword_h = [line.strip() for line in file.readlines()]

[2-3] 토큰화 <hr>

In [111]:
# KoNlpy의 형태소 분석기
from konlpy.tag import Okt

# 인스턴스 생성
okt = Okt()

In [134]:
vocab = {}
for idx in trainDF.index:
    words = okt.morphs(trainDF.loc[idx,'document'])
    for word in words:
        if len(word) >= 2:
            if word not in vocab:
                vocab[word] = 1
            else:
                vocab[word] += 1

In [135]:
# 한글 불용어 제거
for key in list(vocab.keys()):
    if key in stopword_h:
        del vocab[key]

In [137]:
# 빈도수 정렬
sorted_vocab = sorted(vocab.items(), key=lambda x:x[1], reverse=True)

In [139]:
vocabDF = pd.DataFrame(sorted_vocab)
vocabDF

,0,1
0,영화,883
1,너무,354
2,진짜,311
3,최고,302
4,정말,236
...,...,...
10571,한여름,1
10572,남하,1
10573,컾컾,1
10574,으로선,1


[3] 데이터셋용 단어사전/어휘사전 생성<hr>
- 최종 사용할 단어 수
- 특별한 의미의 문자 추가 : '<UNK>', '<PAD>'
- 앞에 번호 뒤에 단어 추가 / 0 : UNK , 1 : PAD

In [140]:
VOCAB_DICT = {0:'<UNK>',1:'<PAD>'}

for i in range(vocabDF.shape[0]):
    VOCAB_DICT[i+2] = vocabDF.iloc[i,0]

In [141]:
VOCAB_DICT

{0: '<UNK>',
 1: '<PAD>',
 2: '영화',
 3: '너무',
 4: '진짜',
 5: '최고',
 6: '정말',
 7: '감동',
 8: '재미',
 9: '쓰레기',
 10: '평점',
 11: '드라마',
 12: '완전',
 13: '별로',
 14: '노잼',
 15: '그냥',
 16: '연기',
 17: '사랑',
 18: '대박',
 19: '최악',
 20: '스토리',
 21: '재밌어요',
 22: '보고',
 23: '좋아요',
 24: '생각',
 25: '없다',
 26: '입니다',
 27: '사람',
 28: '좋은',
 29: '이건',
 30: '재밋',
 31: '하고',
 32: '이영화',
 33: '마지막',
 34: '배우',
 35: '재밌다',
 36: '보다',
 37: '명작',
 38: '감독',
 39: '재밌게',
 40: '처음',
 41: '재미없다',
 42: '같은',
 43: '봤는데',
 44: '재미있어요',
 45: '아깝다',
 46: '다시',
 47: '봐도',
 48: '좋다',
 49: '추강',
 50: '재밌음',
 51: '역시',
 52: '이나',
 53: '없는',
 54: '보기',
 55: '눈물',
 56: '정도',
 57: '없고',
 58: '추천',
 59: '여자',
 60: '인데',
 61: '내용',
 62: '많이',
 63: '실망',
 64: '작품',
 65: '인생',
 66: '재미있다',
 67: '진심',
 68: '주인공',
 69: '는데',
 70: '재미없음',
 71: '이네',
 72: '봤어요',
 73: '한번',
 74: '이렇게',
 75: '짱짱',
 76: '장면',
 77: '으리',
 78: '액션',
 79: '장난',
 80: '결말',
 81: '존나',
 82: '없음',
 83: '가슴',
 84: '합니다',
 85: '봤다',
 86: '아아',
 87: '이야',
 88: '지금'

In [161]:
# 문장을 수치화 진행 즉, 인코딩
test = ['영화', '최고', '사랑']


encoding = []
for tt in test:
    found = 0
    for k, v in VOCAB_DICT.items():
        if v == tt:
            encoding.append(k)
            found = 1
    if not found:
        encoding.append('0')
print(encoding)


[2, 5, 17]


In [167]:
# 문장을 수치화 진행 즉, 인코딩
encoding = [next((k for k, v in VOCAB_DICT.items() if v == tt), 0) for tt in test] 
# 메모리 사용량이 적음 - 제너레이터 : for문 돌리면서 코드가 바뀌어도 그대로 다른 결과 도출 가능
print(encoding)

[2, 5, 17]


In [168]:
# 수치값을 문자열로 변환 진행 즉, 디코딩

decoding = []
for e in encoding:
    decoding.append(VOCAB_DICT.get(e))

print(decoding)

['영화', '최고', '사랑']
